In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

The filetest.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.


# **Static, partial equilibrium model**
### **Production module, example 3:**

*Example 3 defines a partial equilibrium model from data, including nesting structure, and calibration targets. The model is as stylized as possible, ignoring investments, trade and taxes.*

**NB: The module can run with the cobb-douglas specification ($\sigma=1$). However, when this is the case, the sum of $\mu$-parameters should equal 1 (to keep constant returns to scale).**

### **1: Read in data, and nesting structure**

Read in general equilibrium data:

In [2]:
data_folder = os.getcwd()+'\\PE_P\\Example_3'
data = {'Production_v': data_folder+'\\Production_v.xlsx', 'Production_p': data_folder+'\\Production_p.xlsx'}
components = ['domestic']
db_GE = ReadData.read_data.main(data,components=components)

Extract data on sector 's1' from GE data:

In [3]:
s0 = 's1'
db_PE = ReadData.PE_from_GE(db_GE,s0)

Read in nesting:

In [4]:
nt = nesting_tree.nesting_tree(name='test')

*Input-part:*

In [5]:
nt.add_tree(data_folder+'\\nest_s1_in.xlsx',name='s1_in')
read_type = {'1dvars': {'sheets': ['sigma'],'names':{}}, 'vars_panel': {'sheets': {'mu': 2},'names': {}}}
nt.trees['s1_in'].database.read_from_excel(data_folder+'\\nest_s1_in.xlsx',read_type)

*Output-split:*

In [6]:
nt.add_tree(data_folder+'\\nest_s1_out.xlsx',name='s1_out',**{'type_io': 'output', 'type_f': 'CET'})
nt.trees['s1_out'].database.read_from_excel(data_folder+'\\nest_s1_out.xlsx',read_type)

*NB: Note that both Y1,Y2 are used as inputs and outputs. To accomodate for this, we use the feature of temporarily applying new set-names to selected elements of the nesting tree. A dict keeps track of the original names and reverts the names after collecting the nesting trees later on. This feature is called by adding 'temp_namespace' as in the following:*

In [7]:
nt.trees['s1_out'].temp_namespace = {'Y1': 'Y1_out', 'Y2': 'Y2_out'}

### **2: Set up baseline model**

*Compile tree attributes:*

In [8]:
nt.run_all()

*Note that once the aggregate tree has been compiled, the temporary namespace used to have Y1,Y2 as both inputs and outputs are reversed per default - hence the 'temporary' in temporary namespace. If this is not the case, it should be specified through other means than the 'self.temp_namspace' attribute.*

*Initiate production module:*

In [9]:
pm = PE.GPM_STA_PE.production(nt)

In [10]:
pm.df_write(repo=os.getcwd()+'\\V1',export_settings=True)
pm.create_model_instance(name='baseline',repo=work_folder)
pm.df_run(name='baseline')